# Supercharge your Reporting Services - An essential toolkit
## Craig Porteous | [@cporteous](https://twitter.com/cporteous) | [craigporteous.com](https://craigporteous.com) | [github.com/cporteou](https://github.com/cporteou)

## Content Management

### Orphaned Datasources
---

Data sources with no dependent datasets or reports. This is a maintenance script to keep your environment tidy

In [ ]:
SELECT 
    cat.name, 
    cat.Path,
    cat.CreationDate 
FROM ReportServer.dbo.Catalog cat
LEFT JOIN ReportServer.dbo.Datasource ds
ON ds.Link = cat.ItemId
WHERE ds.Link IS NULL
AND cat.Type = 5

### Unused Reports
---
Identify reports that aren't being used. This is dependent on the timeframe of your execution log but you can always tailor this using a historical execution log table.

In [ ]:
SELECT 
	cat.Name,
    cat.Path,
    cat.CreationDate,
    cat.ModifiedDate
FROM 
	ReportServer.dbo.Catalog cat 
WHERE 
	cat.ItemID NOT IN (SELECT DISTINCT ReportID FROM ReportServer.dbo.ExecutionLogStorage)
AND cat.Type = 2 --Report
ORDER BY
	cat.Path

### Top 10 longest running reports
---
Reports that are consistently taking a long time to run. These will be candidates for performance improvements

In [ ]:
SELECT TOP 10 
	CASE
		WHEN LEN(Itempath) =0 THEN 'ReportBuilder'
		ELSE ItemPath
	END AS ReportPath,
	RequestType,
	UserName,
	TimeStart,
	TimeEnd,
	Format,
	((TimeDataRetrieval + TimeProcessing + TimeRendering)/1000) AS Total_Time_secs,
	[Status],
	[RowCount]
FROM ReportServer.dbo.ExecutionLog3
WHERE TimeStart >= GETDATE()- 365
ORDER BY (TimeDataRetrieval + TimeProcessing + TimeRendering) DESC

### Large Report Executions
---
Are users running reports with a significant row count? These may not be an issue in itself but could be combined with the long running report analysis

In [ ]:
SELECT 
	CASE
		WHEN LEN(Itempath) =0 THEN 'ReportBuilder'
		ELSE ItemPath
	END AS ReportPath,
	UserName,
	TimeStart,
	Format,
	[Status],
	[RowCount]
FROM ReportServer.dbo.ExecutionLog3
WHERE [RowCount] >= 100000
-- AND RequestType <> 'Subscription'
AND TimeStart >= GETDATE()-365
ORDER BY TimeStart DESC